In [6]:
# import necessary stuff and python-wrapper of verse
import os
import numpy as np
from scipy.sparse import csr_matrix

from python.wrapper import VERSE

In [7]:
# define path to c++ inplementation of verse and its *.so compiled files
# instantiate verse algorithm object
cpath = os.path.dirname(os.path.realpath('src/verse.cpp'))
verse = VERSE(cpath=cpath)

In [8]:
# construct example adjacency matrix in csr format
row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])

csr_adjacency_matrix = csr_matrix((data, (row, col)), shape=(3, 3))

In [9]:
# define hyper-parameters for verse with personalized page rank as sim_G function
w=None
n_hidden=128
steps=100000
n_neg_samples=3
lr=0.0025
rng_seed=0
n_threads=-1
graph = csr_adjacency_matrix

array([[ -8.24541524e-02,   2.61765365e-02,  -3.44700702e-02,
         -5.02266772e-02,  -1.32048473e-01,   6.45890161e-02,
          2.17394754e-02,  -7.01409429e-02,  -4.95584346e-02,
          1.00026159e-02,   5.28095961e-02,  -3.01387925e-02,
         -1.36691378e-03,   8.13108757e-02,  -8.93724933e-02,
          4.47772108e-02,   6.43548965e-02,   6.39759079e-02,
          8.83735046e-02,  -6.39057681e-02,   1.04005650e-01,
          3.03953625e-02,   6.23095734e-03,   1.31064327e-04,
          2.79673226e-02,  -1.18600884e-02,  -1.19457133e-02,
         -1.13681452e-02,   1.25615709e-02,   1.05606355e-01,
          3.19657139e-02,   1.68593582e-02,  -5.78731894e-02,
          5.94110340e-02,   5.18112108e-02,  -5.01754060e-02,
          2.68568452e-02,  -3.53996153e-03,   4.26222906e-02,
         -2.67554168e-02,  -5.62147796e-02,   2.62326114e-02,
         -8.15185308e-02,   6.64441735e-02,  -5.85792623e-02,
         -2.03871746e-02,   4.42609824e-02,   2.49830447e-02,
        

In [ ]:
# compute node embeddings
# ATTENTION: takes a while until termination and cannot be terminated through restarting kernel
# Need to kill terminal, where jupyter notebook is started in in order to kill c++-process
verse.verse_ppr(graph)

In [4]:
# get dimensionality of matrix described by csr format
graph.shape

(10, 10)